# Schwarzschild Solution to the Einstein's equation

We will rederive the equations from *General Theory of Relativity*, chapter 18, by Dirac. They describe a spherically symmetric solution to the Einstein's equation in vacuum.

We will not try to solve the resulting equations.

This version of the notebook uses functions that explicitly calculate the Christoffel symbols, hence the work is coordinate-system-dependent.

Import the necessary modules.

In [1]:
from sympy import *
from sympy.diffgeom import *
TP = TensorProduct

Define a 4D manifold with a patch and a coordinate chart on which we will work.

In [2]:
m = Manifold('Schwarzschild', 4)
p = Patch('origin', m)
cs = CoordSystem('spherical', p, symbols('t r theta phi', real=True))

In [3]:
m, p, cs

(Schwarzschild, origin, spherical)

Prepare the variables containing the scalar fields and the 1-form fields.

In [4]:
t, r, theta, phi = cs.coord_functions()
t, r, theta, phi

(t, r, theta, phi)

In [5]:
dt, dr, dtheta, dphi = cs.base_oneforms()
dt, dr, dtheta, dphi

(dt, dr, dtheta, dphi)

The most general spherically-symmetric metric has the following form.

In [6]:
f, g = symbols('f, g', cls=Function)
metric = exp(2*f(r))*TP(dt, dt) - exp(2*g(r))*TP(dr, dr) - r**2*TP(dtheta, dtheta) - r**2*sin(theta)**2*TP(dphi, dphi)
metric

exp(2*f(r))*TensorProduct(dt, dt) - exp(2*g(r))*TensorProduct(dr, dr) - sin(theta)**2*r**2*TensorProduct(dphi, dphi) - r**2*TensorProduct(dtheta, dtheta)

The matrix $M$ representing the two-form as a bilinear map $V,U\to V^tMU$ over the column vectors $V$ and $U$ in the canonical basis of the chosen coordinate system is:

In [7]:
twoform_to_matrix(metric)

Matrix([
[exp(2*f(r)),            0,     0,                   0],
[          0, -exp(2*g(r)),     0,                   0],
[          0,            0, -r**2,                   0],
[          0,            0,     0, -sin(theta)**2*r**2]])

Now we will calculate the components in the same basis of the Ricci tensor.

In [8]:
ricci = metric_to_Ricci_components(metric)

In [9]:
ricci = [[simplify(ricci[i][j])
            for j in range(4)] for i in range(4)]


The diagonal components give the equations we are interested in.

In [10]:
ricci[0][0]

(r*Subs(Derivative(f(_xi), _xi), _xi, r)**2 - r*Subs(Derivative(f(_xi), _xi), _xi, r)*Subs(Derivative(g(_xi), _xi), _xi, r) + r*Subs(Derivative(f(_xi), (_xi, 2)), _xi, r) + 2*Subs(Derivative(f(_xi), _xi), _xi, r))*exp(2*f(r) - 2*g(r))/r

In [11]:
ricci[1][1]

-Subs(Derivative(f(_xi), _xi), _xi, r)**2 + Subs(Derivative(f(_xi), _xi), _xi, r)*Subs(Derivative(g(_xi), _xi), _xi, r) - Subs(Derivative(f(_xi), (_xi, 2)), _xi, r) + 2*Subs(Derivative(g(_xi), _xi), _xi, r)/r

In [12]:
ricci[2][2]

(exp(2*g(r)) - r*Subs(Derivative(f(_xi), _xi), _xi, r) + r*Subs(Derivative(g(_xi), _xi), _xi, r) - 1)*exp(-2*g(r))

In [13]:
ricci[3][3]

(exp(2*g(r)) - r*Subs(Derivative(f(_xi), _xi), _xi, r) + r*Subs(Derivative(g(_xi), _xi), _xi, r) - 1)*exp(-2*g(r))*sin(theta)**2

The off-diagonal components are zero.

In [14]:
all(ricci[i][j]==0 for i in range(4) for j in range(4) if i!=j)

True

For completeness we can also check out the Christoffel symbol of 2nd kind. We will print only the non-zero components, and only one of the symmetric components (symmetric in the last two indices).

In [15]:
ch_2nd = metric_to_Christoffel_2nd(metric)
filt = [((i,j,k), simplify(ch_2nd[i][j][k]))
            for i in range(4) for j in range(4) for k in range(j,4)
            if ch_2nd[i][j][k]!=0]

In [16]:
filt[0:3]

[((0, 0, 1), Subs(Derivative(f(_xi), _xi), _xi, r)),
 ((1, 0, 0), exp(2*f(r) - 2*g(r))*Subs(Derivative(f(_xi), _xi), _xi, r)),
 ((1, 1, 1), Subs(Derivative(g(_xi), _xi), _xi, r))]

In [17]:
filt[3:6]

[((1, 2, 2), -exp(-2*g(r))*r),
 ((1, 3, 3), -exp(-2*g(r))*sin(theta)**2*r),
 ((2, 1, 2), 1/r)]

In [18]:
filt[6:9]

[((2, 3, 3), -sin(2*theta)/2), ((3, 1, 3), 1/r), ((3, 2, 3), 1/tan(theta))]

We can also confirm that the Christoffel symbol is symmetric.

In [19]:
all(ch_2nd[k][i][j] == ch_2nd[k][j][i] for k in range(4) for i in range(4) for j in range(4))

True